# Assignment 19

## Dataset Cleaning/Preparing

In [3]:
library(ranger)
library(caret)
library(MB14)
a19 = read.csv("hmeq.csv")
a19$BAD = as.factor(a19$BAD)
names(a19)[1] = "target"
a19 = QuickClean(a19,0)

Loading required package: lattice
Loading required package: ggplot2


## Initial Model Building

In [20]:
splitIndex <- createDataPartition(a19$target, p = .70, list = FALSE, times = 1)
train <- a19[ splitIndex,]
test <- a19[-splitIndex,]
train1 = train[train$target == "1",]
train0 = train[train$target == '0',]
n1 = nrow(train1)
n0 = nrow(train0)
train00 = train0[sample(1:n0,n1),]
train_under = rbind(train00, train1)
rf1 = ranger(target~., data = train_under)
c1 = predict(rf1, data = test)$predictions
cm_orig=confusionMatrix(factor(c1), test$target, positive="1")
cm_orig$table
cm_orig$overall[1]
cm_orig$byClass[11]

          Reference
Prediction   0   1
         0 775  22
         1 186  70

Accuracy 
0.8024691

Balanced Accuracy 
        0.7836606

## Undersampling Model Building

In [24]:
splitIndex <- createDataPartition(a19$target, p = .70, list = FALSE, times = 1)
train <- a19[ splitIndex,]
test <- a19[-splitIndex,]
train1 = train[train$target == "1",]
train0 = train[train$target == '0',]
n1 = nrow(train1)
n0 = nrow(train0)
mc = matrix(0, ncol = 1000, nrow = nrow(test))
for (i in 1:1000)
{
  train00 = train0[sample(1:n0,n1),]
  train_under = rbind(train00, train1)
  rf1 = ranger(target~., data = train_under)
  c0 = predict(rf1, data = test)$predictions
  mc[,i] = c0
}
mc = mc - 1

In [1]:
cc = c(round((rowMeans(mc))))

In [4]:
cm=confusionMatrix(factor(cc), test$target, positive="1")
cm$table
cm$overall[1]
cm$byClass[11]

          Reference
Prediction   0   1
         0 836  12
         1 125  80

Accuracy 
0.8698955

Balanced Accuracy 
        0.8697462